<a href="https://colab.research.google.com/github/darsiltech/scoring-de-clientes/blob/main/Copia_de_scoring_de_clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Scoring de Clientes

Basandome en un dataset real de un empresa financiera tomando el historico de 100 clientes activos hasta estos dias, busco crear una herramienta que me permita un mejor analisis comercial y de riesgo.

Los datos estan en un excel q esta subudo en dive:
https://docs.google.com/spreadsheets/d/11zQYsr3FVpSwXC5dZ-izM1UQqE5OtI6T/edit?usp=drive_link&ouid=115500018994462773408&rtpof=true&sd=true


Importacion de libreria y dataset

In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/100 casos.xlsx")
df.shape

(674, 24)

Tipos de Datos

In [16]:
print('\nLostipos de datos son de tipo:\n',df.dtypes)


Lostipos de datos son de tipo:
 Cod Age                             int64
Agencia                            object
ID_CI                               int64
Solicitud                           int64
Aceptada                   datetime64[ns]
Producto                            int64
Nombre                             object
Tasa                              float64
Cap. solicitado                   float64
Cap. liquido                      float64
Cuotas                              int64
Mto Cuota                           int64
Familia                            object
Edad                                int64
Sexo                               object
Profesión                          object
Empresa                            object
Cant. Solic. Vigentes               int64
Cant. Solic. Canceladas             int64
Departamento                       object
Localidad                          object
Tipo de Venta                      object
Capital en mano                   float64
O

Primeras 5 filas

In [17]:
df.head(5)

,Cod Age,Agencia,ID_CI,Solicitud,Aceptada,Producto,Nombre,Tasa,Cap. solicitado,Cap. liquido,...,Sexo,Profesión,Empresa,Cant. Solic. Vigentes,Cant. Solic. Canceladas,Departamento,Localidad,Tipo de Venta,Capital en mano,Origen
0,9,Bella Unión,7386,8351,2004-08-26,76,PROM JULIO/04 $3000,70.00,3195.0,3000.0,...,MASCULINO,PUBLICO,"ADMINISTRACION NAL, DE CORREOS",1,16,ARTIGAS,BU,NaN,3000.00,NaN
1,9,Bella Unión,7386,13994,2005-03-21,52,"BUEN CLIENTE 13-18 COOP,",63.47,5575.0,5000.0,...,MASCULINO,PUBLICO,"ADMINISTRACION NAL, DE CORREOS",1,16,ARTIGAS,BU,NaN,4104.40,NaN
2,10,Paysandú,11854,15914,2005-05-10,49,"STANDAR 07-12 COOP,",75.80,4500.0,4000.0,...,MASCULINO,PRIVADO,SUCESORES DE PEDRO WYAUX LARDO,0,9,PAYSANDU,PU,NaN,4000.00,NaN
3,9,Bella Unión,7386,25442,2005-11-17,127,PROMO 5000 SET 05,55.00,5225.0,5000.0,...,MASCULINO,PUBLICO,"ADMINISTRACION NAL, DE CORREOS",1,16,ARTIGAS,BU,NaN,2653.43,NaN
4,8,Colonia,10116,26530,2005-12-06,134,PROMO NOV 05 6000,52.20,6270.0,6000.0,...,FEMENINO,PUBLICO,EDUCACION PRIMARIA,0,8,COLONIA,CO,NaN,6000.00,NaN


predecir si un cliente incumplirá en sus pagos

Preparación de Datos
Incluye: ✅ Carga y unión de datos: Como las tablas están relacionadas por ID_CI, debemos unirlas.
✅ Selección de variables relevantes: Debemos elegir características (X) que puedan influir en la variable objetivo (y).
✅ Limpieza y transformación: Manejar valores nulos, codificar variables categóricas y normalizar datos numéricos.
✅ División en entrenamiento y prueba

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Cargar el dataset desde el archivo Excel
file_path = "/content/drive/MyDrive/Colab Notebooks/100 casos.xlsx"
df_dict = pd.read_excel(file_path, sheet_name=None)

# Unir datos en un solo DataFrame
df = df_dict["SOLICITU"].merge(df_dict["CLEARING"], on="ID_CI", how="left")
df = df.merge(df_dict["CLIENTE"], on="ID_CI", how="left")

# Variables predictoras (X) y variable objetivo (y)
X = df[["SolCapital", "SolCuotas", "CliIngLiq", "CleTCons"]]  # Puedes agregar más
y = df["CleTIncum"]  # Variable a predecir

# Eliminar filas con valores nulos
df.dropna(subset=["CleTIncum"], inplace=True)

# Convertir variables categóricas si es necesario
encoder = LabelEncoder()
if df["CleTIncum"].dtype == "object":
    df["CleTIncum"] = encoder.fit_transform(df["CleTIncum"])

# Normalizar datos numéricos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir en datos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# Eliminar filas donde CleTIncum es NaN antes de definir X e y
df = df.dropna(subset=["CleTIncum"])

# Definir variables predictoras (X) y objetivo (y)
X = df[["SolCapital", "SolCuotas", "CliIngLiq", "CleTCons"]]
y = df["CleTIncum"]

# Convertir variable objetivo a numérica si es necesario
encoder = LabelEncoder()
if y.dtype == "object":
    y = encoder.fit_transform(y)

# Normalizar datos numéricos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar si aún quedan NaN en y_train
print("Valores NaN en y_train:", pd.Series(y_train).isna().sum())


Valores NaN en y_train: 0


Entrenar un Modelo de Machine Learning
Usaremos un Random Forest como modelo inicial.

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Crear y entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicción en datos de prueba
y_pred = model.predict(X_test)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


Precisión del modelo: 0.98
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1733
         1.0       0.72      0.67      0.69        27
         2.0       0.91      0.45      0.61        22
         3.0       0.00      0.00      0.00         8

    accuracy                           0.98      1790
   macro avg       0.65      0.53      0.57      1790
weighted avg       0.98      0.98      0.98      1790



La Regresión Logística es útil cuando queremos predecir una variable categórica binaria, como si un cliente incumplirá o no en sus pagos.

Preparación de Datos
Vamos a predecir si un cliente tendrá incumplimientos (CleTIncum).

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Cargar el dataset desde el archivo Excel
file_path = "/content/drive/MyDrive/Colab Notebooks/100 casos.xlsx"
df_dict = pd.read_excel(file_path, sheet_name=None)

# Unir datos de interés en un solo DataFrame
df = df_dict["SOLICITU"].merge(df_dict["CLEARING"], on="ID_CI", how="left")
df = df.merge(df_dict["CLIENTE"], on="ID_CI", how="left")

# Seleccionar variables predictoras (X) y variable objetivo (y)
X = df[["SolCapital", "SolCuotas", "CliIngLiq", "CleTCons"]]  # Puedes agregar más
y = df["CleTIncum"]  # Variable a predecir (1 = Incumplió, 0 = No incumplió)

# Eliminar filas con valores nulos
df.dropna(subset=["CleTIncum"], inplace=True)

# Convertir variable objetivo a numérica si es necesario
encoder = LabelEncoder()
if y.dtype == "object":
    y = encoder.fit_transform(y)

# Normalizar datos numéricos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir en datos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
# Eliminar filas donde CleTIncum es NaN antes de definir X e y
df = df.dropna(subset=["CleTIncum"])

# Definir variables predictoras (X) y objetivo (y)
X = df[["SolCapital", "SolCuotas", "CliIngLiq", "CleTCons"]]
y = df["CleTIncum"]

# Convertir variable objetivo a numérica si es necesario
encoder = LabelEncoder()
if y.dtype == "object":
    y = encoder.fit_transform(y)

# Normalizar datos numéricos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar si aún quedan NaN en y_train
print("Valores NaN en y_train:", pd.Series(y_train).isna().sum())


Valores NaN en y_train: 0


Entrenar el Modelo de Regresión Logística

In [25]:
# Crear y entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


Precisión del modelo: 0.97
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      1733
         1.0       0.00      0.00      0.00        27
         2.0       0.00      0.00      0.00        22
         3.0       0.00      0.00      0.00         8

    accuracy                           0.97      1790
   macro avg       0.24      0.25      0.25      1790
weighted avg       0.94      0.97      0.95      1790



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
